In [9]:
SEASON = '2006-07'

In [10]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Dirk Nowitzki,DAL,PF,113.117783,28,78,78,36.2,8.6,17.2,...,28.9,11.8,4.5,16.3,0.278,7.2,1.2,8.3,7.4,67.0
2,LeBron James,CLE,SF,102.599030,22,78,78,40.9,9.9,20.8,...,31.0,8.0,5.7,13.7,0.206,5.9,2.2,8.1,8.1,50.0
3,Tim Duncan,SAS,C,89.113959,30,80,80,34.1,7.7,14.1,...,27.9,6.3,6.8,13.0,0.230,3.8,3.2,7.0,6.2,58.0
4,Kobe Bryant,LAL,SG,83.877572,28,77,77,40.8,10.6,22.8,...,33.6,10.8,2.2,13.0,0.199,6.4,-0.5,5.9,6.1,42.0
5,Steve Nash,PHO,PG,80.654647,32,76,76,35.3,6.8,12.8,...,22.9,10.8,1.7,12.6,0.225,6.7,-0.8,5.9,5.3,61.0
6,Dwyane Wade,MIA,SG,74.977260,25,51,50,37.9,9.3,18.9,...,34.7,5.9,2.9,8.8,0.219,6.3,1.8,8.2,4.9,44.0
7,Shawn Marion,PHO,SF,71.435843,28,80,80,37.6,7.0,13.4,...,19.3,6.7,5.3,12.0,0.191,2.4,1.9,4.4,4.8,61.0
8,Tracy McGrady,HOU,SG,70.585475,27,71,71,35.8,9.0,20.8,...,35.0,4.1,4.5,8.6,0.163,4.9,1.2,6.1,5.2,52.0
9,Manu Ginóbili,SAS,SG,69.714324,29,75,36,27.5,5.3,11.4,...,27.1,6.3,4.3,10.6,0.246,5.3,2.5,7.8,5.1,58.0
10,Gilbert Arenas,WAS,PG,67.490872,25,74,73,39.8,8.7,20.9,...,31.4,9.3,1.5,10.8,0.177,5.5,-0.8,4.7,5.0,41.0
